# Implementation
Author: Jørgen S. Dokken


## Creating the mesh
To create the computational geometry, we use the python-API of [GMSH](http://gmsh.info/). We start by import the gmsh-module, and initalizing it.

In [1]:
import gmsh
gmsh.initialize()

The next step is to create the membrane and starting the computations by the GMSH CAD kernel, to generate the relevant underlying data structures. The arguments into `addDisk` is the x, y and z coordinate of the center of the circle, while the to last argument is the x-radius and y-radius.

In [2]:
membrane = gmsh.model.occ.addDisk(0, 0, 0, 1, 1)
gmsh.model.occ.synchronize()

The next step is to make the membrane a physical surface, such that it is recognized by gmsh when generating the mesh. As a surface is a two-dimensional entity, we add two as the first argument, and the entity tag of the as the second argument. The last argument is the physical tag. In a later demo, we will get into when this tag matters.

In [3]:
gdim = 2
status = gmsh.model.addPhysicalGroup(gdim, [membrane], 1)

Finally, we generate the two-dimensional mesh. We make the output in gmsh visible by setting the `Terminal` option

In [4]:
gmsh.model.mesh.generate(gdim)

We will import the GMSH-mesh directly from GMSH, using the approach in Section 2 of [A GMSH tutorial for dolfinx](http://jsdokken.com/converted_files/tutorial_gmsh.html). To make sure this runs in parallel and  serial, we will read in the mesh on one processor, and let dolfin-X distribute the mesh data among the processros.

In [8]:
from dolfinx.io import extract_gmsh_geometry,extract_gmsh_topology_and_markers, ufl_mesh_from_gmsh
from mpi4py import MPI
from warnings import filterwarnings

filterwarnings("ignore")
if MPI.COMM_WORLD.rank == 0:
    # Get mesh geometry
    geometry_data = extract_gmsh_geometry(gmsh.model)
    # Get mesh topology for each element
    topology_data = extract_gmsh_topology_and_markers(gmsh.model)

The topology data is a dictionary, where the key is the gmsh cell type (an integer). Each key accesses a dictionary with the topology data and corresponding topology markers. As this mesh only contains one cell type (triangles), as we did not mark any facets, we do not need to loop over the keys of this dictionary, only extract the first one.

In [6]:
if MPI.COMM_WORLD.rank == 0:
    # Extract the cell type and number of nodes per cell and broadcast
    # it to the other processors 
    gmsh_cell_type = list(topology_data.keys())[0]    
    properties = gmsh.model.mesh.getElementProperties(gmsh_cell_type)
    name, dim, order, num_nodes, local_coords, _ = properties
    cells = topology_data[gmsh_cell_type]["topology"]
    cell_id, num_nodes = MPI.COMM_WORLD.bcast([gmsh_cell_type, num_nodes], root=0)
else:        
    cell_id, num_nodes = MPI.COMM_WORLD.bcast([None, None], root=0)
    cells, geometry_data = numpy.empty([0, num_nodes]), numpy.empty([0, gdim])

As we have now broadcased all the information required to distribute the mesh in parallel

In [7]:
from dolfinx.cpp.io import perm_gmsh
from dolfinx.cpp.mesh import to_type
from dolfinx.mesh import create_mesh

# Permute topology data from MSH-ordering to dolfinx-ordering
ufl_domain = ufl_mesh_from_gmsh(cell_id, gdim)
gmsh_cell_perm = perm_gmsh(to_type(str(ufl_domain.ufl_cell())), num_nodes)
cells = cells[:, gmsh_cell_perm]

# Create distributed mesh
mesh = create_mesh(MPI.COMM_WORLD, cells, geometry_data[:, :gdim], ufl_domain)


## Defining the load
Authors: Hans Petter Langtangen and Anders Logg.

Adapted to dolfin-X: Jørgen S. Dokken